In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/processedBuildingLabels/summarydata/AOI_1_RIO_polygons_solution_3band.csv')
print(df.shape)
df.head()

(255881, 4)


,ImageId,BuildingId,PolygonWKT_Pix,PolygonWKT_Geo
0,AOI_1_RIO_img5792,1,POLYGON ((408.210510077716208 39.5679995656149...,POLYGON ((-43.541791399999966 -22.876368899999...
1,AOI_1_RIO_img5792,2,POLYGON ((389.833864435032297 42.6174620222885...,POLYGON ((-43.541873799999962 -22.876382599999...
2,AOI_1_RIO_img5792,3,POLYGON ((242.119589264420313 -0.0000059554636...,POLYGON ((-43.542536143743767 -22.876191136973...
3,AOI_1_RIO_img5792,4,POLYGON ((311.733120444117787 216.147456877115...,"POLYGON ((-43.542224 -22.8771622 0,-43.5421641..."
4,AOI_1_RIO_img5792,5,POLYGON ((350.582776650603819 112.220886068182...,POLYGON ((-43.542049799999972 -22.876695299999...


In [3]:
print(np.count_nonzero(df['PolygonWKT_Pix'] == 'POLYGON EMPTY'))

2630


In [4]:
df = df[df['PolygonWKT_Pix'] != 'POLYGON EMPTY']
df.shape

(253251, 4)

# Train-Val Split

In [5]:
import random
random.seed(5678)
image_ids = sorted(df['ImageId'].unique())
print(len(image_ids))
random.shuffle(image_ids)

4310


In [6]:
split_ratio = 0.8
num_train_ids = round(len(image_ids) * split_ratio)
train_ids = image_ids[0:num_train_ids]
print('len(train_ids): {}'.format(len(train_ids)))
val_ids = image_ids[num_train_ids:]
print('len(val_ids): {}'.format(len(val_ids)))

df['val'] = df['ImageId'].isin(val_ids)

len(train_ids): 3448
len(val_ids): 862


In [7]:
# From CrowdAI val: 60317 all, 1820 small (0.03).
small_frac = 0.03
small_train_ids = train_ids[:int(small_frac*len(train_ids))]
print('len(small_train_ids): {}'.format(len(small_train_ids)))
small_val_ids = val_ids[:int(small_frac*len(val_ids))]
print('len(small_val_ids): {}'.format(len(small_val_ids)))

len(small_train_ids): 103
len(small_val_ids): 25


In [8]:
train_df = df[df['ImageId'].isin(train_ids)]
print('len(train_df): {}'.format(len(train_df)))
val_df = df[df['ImageId'].isin(val_ids)]
print('len(val_df): {}'.format(len(val_df)))

len(train_df): 203245
len(val_df): 50006


In [9]:
small_train_df = df[df['ImageId'].isin(small_train_ids)]
print('len(small_train_df): {}'.format(len(small_train_df)))
small_val_df = df[df['ImageId'].isin(small_val_ids)]
print('len(small_val_df): {}'.format(len(small_val_df)))

len(small_train_df): 5749
len(small_val_df): 1026


# EDA (pixel coordinates)

In [10]:
print(train_df.shape)
train_df.head()

(203245, 5)


,ImageId,BuildingId,PolygonWKT_Pix,PolygonWKT_Geo,val
0,AOI_1_RIO_img5792,1,POLYGON ((408.210510077716208 39.5679995656149...,POLYGON ((-43.541791399999966 -22.876368899999...,False
1,AOI_1_RIO_img5792,2,POLYGON ((389.833864435032297 42.6174620222885...,POLYGON ((-43.541873799999962 -22.876382599999...,False
2,AOI_1_RIO_img5792,3,POLYGON ((242.119589264420313 -0.0000059554636...,POLYGON ((-43.542536143743767 -22.876191136973...,False
3,AOI_1_RIO_img5792,4,POLYGON ((311.733120444117787 216.147456877115...,"POLYGON ((-43.542224 -22.8771622 0,-43.5421641...",False
4,AOI_1_RIO_img5792,5,POLYGON ((350.582776650603819 112.220886068182...,POLYGON ((-43.542049799999972 -22.876695299999...,False


In [11]:
# Building counts by image?
train_df.groupby('ImageId').count().describe()['BuildingId']

count    3448.000000
mean       58.945766
std        60.207539
min         1.000000
25%         7.000000
50%        35.000000
75%       104.000000
max       305.000000
Name: BuildingId, dtype: float64

In [12]:
from shapely.wkt import loads
polygons = train_df['PolygonWKT_Pix'].apply(loads)

In [13]:
# Building areas?
train_df['areas'] = [polygon.area for polygon in polygons]
train_df['areas'].describe()

/home/yoninachmany/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    203245.000000
mean        436.921027
std        1072.827927
min           0.000001
25%         190.154617
50%         329.352779
75%         510.826394
max      136116.622575
Name: areas, dtype: float64

In [14]:
# Building lengths?
train_df['length'] = [polygon.length for polygon in polygons]
train_df['length'].describe()

/home/yoninachmany/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    203245.000000
mean         81.340062
std          44.609635
min           0.005249
25%          58.518336
50%          76.581172
75%          96.259976
max        2219.464081
Name: length, dtype: float64

In [15]:
bboxes = [polygon.bounds for polygon in polygons]

In [16]:
# Building widths?
train_df['widths'] = [bbox[2] - bbox[0] for bbox in bboxes]
train_df['widths'].describe()

/home/yoninachmany/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    203245.000000
mean         25.289497
std          14.419576
min           0.000428
25%          17.247492
50%          23.611475
75%          30.894192
max         439.000006
Name: widths, dtype: float64

In [17]:
# Buildings heights?
train_df['heights'] = [bbox[3] - bbox[1] for bbox in bboxes]
train_df['heights'].describe()

/home/yoninachmany/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


count    203245.000000
mean         23.917856
std          13.689699
min           0.001529
25%          16.288957
50%          22.390246
75%          29.388162
max         406.000004
Name: heights, dtype: float64

In [18]:
train_df.head()

,ImageId,BuildingId,PolygonWKT_Pix,PolygonWKT_Geo,val,areas,length,widths,heights
0,AOI_1_RIO_img5792,1,POLYGON ((408.210510077716208 39.5679995656149...,POLYGON ((-43.541791399999966 -22.876368899999...,False,515.444317,99.571710,30.107368,25.130242
1,AOI_1_RIO_img5792,2,POLYGON ((389.833864435032297 42.6174620222885...,POLYGON ((-43.541873799999962 -22.876382599999...,False,992.831749,145.971933,34.166288,42.336333
2,AOI_1_RIO_img5792,3,POLYGON ((242.119589264420313 -0.0000059554636...,POLYGON ((-43.542536143743767 -22.876191136973...,False,3.904188,15.998015,7.522984,1.037936
3,AOI_1_RIO_img5792,4,POLYGON ((311.733120444117787 216.147456877115...,"POLYGON ((-43.542224 -22.8771622 0,-43.5421641...",False,315.181531,73.780407,16.882428,24.907653
4,AOI_1_RIO_img5792,5,POLYGON ((350.582776650603819 112.220886068182...,POLYGON ((-43.542049799999972 -22.876695299999...,False,324.217211,75.187899,16.525600,25.464124


# Annotations

In [19]:
import os

TRAIN_DIRECTORY = "data/train"
if not os.path.exists(TRAIN_DIRECTORY):
    os.mkdir(TRAIN_DIRECTORY)
    
VAL_DIRECTORY = "data/train"
if not os.path.exists(VAL_DIRECTORY):
    os.mkdir(VAL_DIRECTORY)

In [20]:
categories = [{'id': 100, 'name': 'building', 'supercategory': 'building'}]

In [21]:
info = \
{'about': ' Challenge 1 - Rio De Janeiro Building Footprints',
 'contributor': 'spacenet.ai',
 'description': 'The first SpaceNet challenge in 2016 aimed to extract building footprints from the DigitalGlobe WorldView 2 satellite imagery at 50cm resolution.',
 'url': 'https://spacenet.ai/spacenet-buildings-dataset-v1',
 'version': '1.0',
 'year': 2016}

In [22]:
train_annotations = {"info": info, "categories": categories}
val_annotations = {"info": info, "categories": categories}
small_train_annotations = {"info": info, "categories": categories}
small_val_annotations = {"info": info, "categories": categories}

In [23]:
import shutil
from PIL import Image
def get_images(image_ids, split, copy=False):
    images = []
    for image_id in image_ids:
        id = int(image_id.replace('AOI_1_RIO_img', ''))
        file_name = '3band_{}.tif'.format(image_id)
        
        in_file_path = 'data/processedBuildingLabels/3band/{}'.format(file_name)
        im = Image.open(in_file_path)
        width, height = im.size

        image = {'file_name': file_name, 'height': height, 'id': id, 'width': width}

        if copy:
            out_file_path = 'data/{}/images/{}'.format(split, file_name)
            shutil.copy2(in_file_path, out_file_path)
        
        images.append(image)
    return images

In [24]:
train_annotations["images"] = get_images(train_ids, 'train')
val_annotations["images"] = get_images(val_ids, 'val')
small_train_annotations["images"] = get_images(small_train_ids, 'train')
small_val_annotations["images"] = get_images(small_val_ids, 'val')

In [25]:
import matplotlib.pyplot as plt
def get_means(image_ids):
    all_means = []
    for i, image_id in enumerate(image_ids):        
        file_path = 'data/processedBuildingLabels/3band/3band_{}.tif'.format(image_id)
        img = plt.imread(file_path)
        means = img.reshape(-1, 3).mean(axis=0)
        all_means.append(means)
    all_means = np.array(all_means)
    return all_means.mean(axis=0)

In [26]:
train_means = get_means(train_ids)
train_means

array([81.16231469, 86.53528546, 64.72005973])

In [27]:
def get_annotations(df):
    annotations = []
    for id, (image_id, polygonwkt_pix) in enumerate(zip(df['ImageId'], df['PolygonWKT_Pix'])):
        image_id = int(image_id.replace('AOI_1_RIO_img', ''))        
        
        polygon = loads(polygonwkt_pix)
        segmentation = [[coord for coords in list(polygon.exterior.coords) for coord in coords[:2]]]
        
        area = polygon.area
        bounds = polygon.bounds
        bbox = [bounds[0], bounds[1], bounds[2]-bounds[0], bounds[3]-bounds[1]]
        
        category_id = 100
        iscrowd = 0
        
        annotation = {"id": id, "image_id": image_id, "segmentation": segmentation, "area": area, "bbox": bbox, "category_id": category_id, "iscrowd": iscrowd}
        annotations.append(annotation)
    return annotations

In [28]:
train_annotations["annotations"] = get_annotations(train_df)
val_annotations["annotations"] = get_annotations(val_df)
small_train_annotations["annotations"] = get_annotations(small_train_df)
small_val_annotations["annotations"] = get_annotations(small_val_df)

In [29]:
import json

TRAIN_ANNOTATIONS_PATH = "data/train/annotation.json"
TRAIN_ANNOTATIONS_SMALL_PATH = "data/train/annotation-small.json"

VAL_ANNOTATIONS_PATH = "data/val/annotation.json"
VAL_ANNOTATIONS_SMALL_PATH = "data/val/annotation-small.json"

with open(TRAIN_ANNOTATIONS_PATH, 'w') as f:
    json.dump(train_annotations, f)

with open(TRAIN_ANNOTATIONS_SMALL_PATH, 'w') as f:
    json.dump(small_train_annotations, f)

with open(VAL_ANNOTATIONS_PATH, 'w') as f:
    json.dump(val_annotations, f)

with open(VAL_ANNOTATIONS_SMALL_PATH, 'w') as f:
    json.dump(small_val_annotations, f)